**Last week, you were introduced to model testing and model selection for a regression task using the framework of cross-validation. In this lab, we'll experiment a bit more with model selection, but this time, we'll be focusing on the task of classification. We'll work with the dataset "heart_failure_lab", which contains several information regarding a patient's health, and whether he/she suffered from an heart failure. The task here is trying to find the best model that can predict an heart failure using the patient's information at hand. To this end, you'll be introduced to several models, that you may consider as "black boxes" (at this stage, you're not asked to understand how the models work under the hood).**

**Import the necessary libraries**

In [1]:
import numpy as np 
import pandas as pd 
from sklearn.impute import SimpleImputer
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split, cross_validate, LeaveOneOut, GridSearchCV, RandomizedSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler, RobustScaler
from sklearn.neighbors import KNeighborsClassifier
from sklearn.pipeline import make_pipeline 
from sklearn.compose import ColumnTransformer

**1) Load the dataframe, get its general information and change the data type of the columns 'anaemia', 'diabetes', 'high_blood_pressure', 'sex', smoking' to categorical.**

In [2]:
file = './heart_failure_lab.csv'
df = pd.read_csv(file, index_col=0)
df = df.astype({'anaemia' : 'category', 'diabetes': 'category', 'high_blood_pressure' : 'category', 'sex' : 'category', 'smoking' : 'category'})
print(df.info())
print(df.head())
print(df.isna().sum())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 299 entries, 0 to 298
Data columns (total 13 columns):
 #   Column                    Non-Null Count  Dtype   
---  ------                    --------------  -----   
 0   age                       294 non-null    float64 
 1   anaemia                   291 non-null    category
 2   creatinine_phosphokinase  292 non-null    float64 
 3   diabetes                  287 non-null    category
 4   ejection_fraction         296 non-null    float64 
 5   high_blood_pressure       291 non-null    category
 6   platelets                 294 non-null    float64 
 7   serum_creatinine          290 non-null    float64 
 8   serum_sodium              296 non-null    float64 
 9   sex                       292 non-null    category
 10  smoking                   293 non-null    category
 11  time                      299 non-null    int64   
 12  DEATH_EVENT               292 non-null    float64 
dtypes: category(5), float64(7), int64(1)
memory usage:

**2) Remove the entire row wherever the corresponding column 'DEATH_EVENT contains a missing value. Change the datatype of the column 'DEATH_EVENT' to integer.**

In [3]:
idx = df[df['DEATH_EVENT'].isnull()].index
df = df.drop(idx, axis=0)
df = df.astype({'DEATH_EVENT' : 'int'})
print(df.info())
print(df.isna().sum())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 292 entries, 0 to 298
Data columns (total 13 columns):
 #   Column                    Non-Null Count  Dtype   
---  ------                    --------------  -----   
 0   age                       287 non-null    float64 
 1   anaemia                   284 non-null    category
 2   creatinine_phosphokinase  285 non-null    float64 
 3   diabetes                  281 non-null    category
 4   ejection_fraction         289 non-null    float64 
 5   high_blood_pressure       284 non-null    category
 6   platelets                 287 non-null    float64 
 7   serum_creatinine          283 non-null    float64 
 8   serum_sodium              289 non-null    float64 
 9   sex                       285 non-null    category
 10  smoking                   286 non-null    category
 11  time                      292 non-null    int64   
 12  DEATH_EVENT               292 non-null    int64   
dtypes: category(5), float64(6), int64(2)
memory usage:

**3) Replace categorical missing values by the most frequent occurence of the corresponding column. For continuous missing values, replace them by the mean. Create a new dataframe 'df_new' which does not contain the variable 'time'.**

In [4]:
imp_cont = SimpleImputer(missing_values=np.nan, strategy='mean')
imp_cat = SimpleImputer(missing_values=np.nan, strategy='most_frequent')

cat_columns = df.select_dtypes(include=['category']).columns
cont_columns = df.select_dtypes(exclude=['category', 'int']).columns

df[cat_columns] = imp_cat.fit_transform(df[cat_columns])
df[cont_columns] = imp_cont.fit_transform(df[cont_columns])

print(df.isna().sum())

age                         0
anaemia                     0
creatinine_phosphokinase    0
diabetes                    0
ejection_fraction           0
high_blood_pressure         0
platelets                   0
serum_creatinine            0
serum_sodium                0
sex                         0
smoking                     0
time                        0
DEATH_EVENT                 0
dtype: int64


In [5]:
df_new = df.drop("time", axis=1)

**4) Fit a model to predict the target variale 'DEATH_EVENT' given the remaining columns of 'df_new'. To this end, use a SVC, a RandomForestClassifier, and a LogisticRegression. Fit the model using a 10 folds cross-validation, and report the mean training and test accuracy across each folds. What are your conclusions ?**  

In [15]:
X = df_new.drop("DEATH_EVENT", axis=1)
y = df_new.DEATH_EVENT

model = SVC(gamma='auto')
#model = RandomForestClassifier()
#model = LogisticRegression()

cv_results = cross_validate(model, X, y, cv=10, scoring=['accuracy'], return_train_score=True)

train_acc = cv_results['train_accuracy']
test_acc = cv_results['test_accuracy']

mean_train_acc = train_acc.mean()
mean_test_acc = test_acc.mean()

print('Train accuracy : {}'.format(mean_train_acc))
print('Test accuracy : {}'.format(mean_test_acc))

'''
Both the SVC and the RandomForestClassifier show a high score on the training data, and a much lower performance on the test sets. This suggests 
that the model overfits the training data (i.e. the models start to model irrelevant patterns in the training data, which do not generalize well to unseen 
data). This indicate that the model has high variance. On the contrary, the LogisticRegression shows low performance on both the training and test data, which 
is a sign of underfitting (the model is actually not complex enough to explain the data). Underfitting is associated to a model having high bias. 
'''

Train accuracy : 1.0
Test accuracy : 0.6781609195402301


'\nBoth the SVC and the RandomForestClassifier show a high score on the training data, and a much lower performance on the test sets. This suggests \nthat the model overfits the training data (i.e. the models start to model irrelevant patterns in the training data, which do not generalize well to unseen \ndata). This indicate that the model has high variance. On the contrary, the LogisticRegression shows low performance on both the training and test data, which \nis a sign of underfitting (the model is actually not complex enough to explain the data). Underfitting is associated to a model having high bias. \n'

**5) Like last week, grid search is an useful procedure when trying to find the best subset of hyper-parameters fo a model. Go check the documentation of the class RandomForestClassifier(), and perform a grid search on a given range of selected hyper-parameters. Set the number of folds to 10, and evaluate on the accuracy. Report the best subset of hyper-parameters, and the best score.**

In [7]:
clf = RandomForestClassifier()

param_grid = { 
    'n_estimators': [10, 20, 30],
    'max_features': ['auto', 'sqrt'],
    'max_depth' : [4,5],
    'criterion' :['gini', 'entropy']}

grid = GridSearchCV(estimator=clf, param_grid=param_grid, cv=10, scoring='accuracy')
grid.fit(X, y)

best_params = grid.best_params_
best_test_score = grid.cv_results_['mean_test_score'].max()

print('Best set of hyper-parameters : {}'.format(best_params))
print('Best test accuracy : {}'.format(best_test_score))

Best set of hyper-parameters : {'criterion': 'entropy', 'max_depth': 4, 'max_features': 'auto', 'n_estimators': 30}
Best test accuracy : 0.7497701149425288


**6) Scaling the variables sometimes helps the classifier in its predictions. Check the class StandardScaler() and create a pipeline (using the method 'make_pipeline') tha combines both the scaler and a SVC(). Be careful, the scaler must only be applied to continuous variables. To this end, check the class ColumnTransformer(), and how it can be used to apply a scaler to only a set of specified columns. Use this papeline this perform the same grid search as before. Repeat the experiment using a RobustScaler().** 

**Do the performance change before and after scaling ?**

In [14]:
columns = df_new.select_dtypes(include=['float']).columns

clf = SVC()

param_grid = { 
    'kernel': ['poly', 'rbf'],
    'degree': [1,2,3]
}

grid = GridSearchCV(estimator=clf, param_grid=param_grid, cv=10, scoring='accuracy')
grid.fit(X,y)

best_params = grid.best_params_
best_test_score = grid.cv_results_['mean_test_score'].max()

print('Best set of hyper-parameters : {}'.format(best_params))
print('Best test accuracy : {}'.format(best_test_score))


Best set of hyper-parameters : {'degree': 1, 'kernel': 'poly'}
Best test accuracy : 0.6781609195402301


In [16]:
columns = df_new.select_dtypes(include=['float']).columns

ct = ColumnTransformer([
        ('num_transformer', StandardScaler(), columns)
    ], remainder='passthrough')

clf = SVC()

pipeline = make_pipeline(ct, clf)

param_grid = { 
    'svc__kernel': ['poly', 'rbf'],
    'svc__degree': [1,2,3]
}

grid = GridSearchCV(estimator=pipeline, param_grid=param_grid, cv=10, scoring='accuracy')
grid.fit(X,y)

best_params = grid.best_params_
best_test_score = grid.cv_results_['mean_test_score'].max()

print('Best set of hyper-parameters : {}'.format(best_params))
print('Best test accuracy : {}'.format(best_test_score))

Best set of hyper-parameters : {'svc__degree': 1, 'svc__kernel': 'poly'}
Best test accuracy : 0.7262068965517241


In [18]:
ct = ColumnTransformer([
        ('num_transformer', RobustScaler(), columns)
    ], remainder='passthrough')

clf = SVC()

pipeline = make_pipeline(ct, clf)

param_grid = { 
    'svc__kernel': ['poly', 'rbf'],
    'svc__degree': [1,2,3]
}

grid = GridSearchCV(estimator=pipeline, param_grid=param_grid, cv=10, scoring='accuracy')
grid.fit(X,y)

best_params = grid.best_params_
best_test_score = grid.cv_results_['mean_test_score'].max()

print('Best set of hyper-parameters : {}'.format(best_params))
print('Best test accuracy : {}'.format(best_test_score))

Best set of hyper-parameters : {'svc__degree': 1, 'svc__kernel': 'rbf'}
Best test accuracy : 0.7535632183908046


Scaling the data ensures that all variables have the same scale. If our dataset, you might have noticed that the scales of 'platelets' and 'ejection_fraction' have orders of magnitude of difference between them. By scaling both variables using a Standard Scaler, we transform them such that their distributions have mean 0 and standard deviation of 1, i.e. they now have the same scale. 

But why is it necessary ? Some models, and especially distance-based models like SVC and KNN that use the distance between two data points to compute their similarity, are greatly affected by highly different scales. As such, to ensure that all features contribute equally to the model, we scale our variables beforehand. 

In the experiments above, we observe that scaling the variables indeed helped the SVC in making its predictions, with the Robust Scaler yielding better performance than the Standard Scaler. 


**7) Let's now see which model amongst a logistic regression, a random forest, a SVC and a multi-layer perceptron performs the best on predicting an heart failure. For each model, perform a grid search cross-validation on a selected subset of hyper-parameters, and report the best model amongst the above. Check the documentation of LogisticRegression(), RandomForestClassifier(), SVC() and MLPClassiffier() to choose the hyper-parameters. Use the same pipeline as before, and evaluate on the accuracy using a 10 folds cross-validation. Report the best model and the best score.**

In [ ]:
import warnings
warnings.filterwarnings("ignore")

logistic = LogisticRegression(solver='saga', max_iter=1000)
forest = RandomForestClassifier()
svc = SVC() 
mlp = MLPClassifier(max_iter=1000)

param_grid_log = {
    'logisticregression__penalty' : ['l2', 'none'],
     'logisticregression__fit_intercept' : [True, False]    
}

param_grid_forest = { 
    'randomforestclassifier__n_estimators': [10, 50, 200],
    'randomforestclassifier__max_features': ['auto', 'sqrt'],
    'randomforestclassifier__max_depth' : [4,5],
    'randomforestclassifier__criterion' :['gini', 'entropy']
}

param_grid_svc = { 
    'svc__kernel': ['linear', 'poly', 'rbf'],
    'svc__degree': [1,2,3],
    'svc__gamma' : ['scale', 'auto'],
}

param_grid_mlp = { 
    'mlpclassifier__hidden_layer_sizes': [(10,), (50,), (100,)],
    'mlpclassifier__activation': ['identity', 'tanh', 'relu'],
    'mlpclassifier__learning_rate' : ['constant', 'invscaling', 'adaptive'],
}

estimators = [logistic, forest, svc, mlp]
grids = [param_grid_log, param_grid_forest, param_grid_svc, param_grid_mlp]
best_results_list = []
best_params_list = []
for i, clf in enumerate(estimators):
  pipeline = make_pipeline(ct, clf)
  print(clf)
  grid = GridSearchCV(estimator=pipeline, param_grid=grids[i], cv=10, scoring='accuracy')
  grid.fit(X, y)
  best_params_list.append(grid.best_params_)
  best_results_list.append(grid.cv_results_['mean_test_score'].max())


print('Best set of hyper-parameters : {}'.format(best_params_list.index(np.argmax(best_results_list))))
print('Best test accuracy : {}'.format(max(best_results_list)))



LogisticRegression(max_iter=1000, solver='saga')
RandomForestClassifier()


KeyboardInterrupt: ignored

**8) As you might have noticed, grid search can quickly take very long to compute when the hyper-parameters space on which to search becomes large. Another approach, that trades precision in the solution for lower runtime, is called Random Search. In a Random Search, only a defined number of hyper-parameters' subsets are selected randomly and used to fit the model, which considerably fastens the procedure. Perform the same experiment as above, but use the class RandomizedSearchCV() this time. Set the number of subsets to try to 5.**

In [ ]:
import warnings
warnings.filterwarnings("ignore")

logistic = LogisticRegression(solver='saga')
forest = RandomForestClassifier()
svc = SVC() 
mlp = MLPClassifier()

param_grid_log = {
    'logisticregression__penalty' : ['l2', 'none'],
     'logisticregression__fit_intercept' : [True, False]    
}

param_grid_forest = { 
    'randomforestclassifier__n_estimators': [10, 50, 200],
    'randomforestclassifier__max_features': ['auto', 'sqrt'],
    'randomforestclassifier__max_depth' : [4,5],
    'randomforestclassifier__criterion' :['gini', 'entropy']
}

param_grid_svc = { 
    'svc__kernel': ['linear', 'poly', 'rbf'],
    'svc__degree': [1,2,3],
    'svc__gamma' : ['scale', 'auto'],
}

param_grid_mlp = { 
    'mlpclassifier__hidden_layer_sizes': [(10,), (50,), (100,)],
    'mlpclassifier__activation': ['identity', 'tanh', 'relu'],
    'mlpclassifier__learning_rate' : ['constant', 'invscaling', 'adaptive'],
}

estimators = [logistic, forest, svc, mlp]
grids = [param_grid_log, param_grid_forest, param_grid_svc, param_grid_mlp]
best_results_list = []
best_params_list = []
best_estimators_list = []
for i, clf in enumerate(estimators):
  print(clf)
  pipeline = make_pipeline(ct, clf)
  grid = RandomizedSearchCV(estimator=pipeline, param_distributions=grids[i], cv=10, scoring='accuracy', n_iter=5)
  grid.fit(X, y)
  best_params_list.append(grid.best_params_)
  best_results_list.append(grid.cv_results_['mean_test_score'].max())
  best_estimators_list.append(grid.best_estimator_)

LogisticRegression(solver='saga')
RandomForestClassifier()
SVC()
MLPClassifier()


In [ ]:
print('Best set of hyper-parameters : {}'.format(best_params_list[np.argmax(best_results_list)]))
print('Best test accuracy : {}'.format(max(best_results_list)))

Best set of hyper-parameters : {'mlpclassifier__learning_rate': 'invscaling', 'mlpclassifier__hidden_layer_sizes': (50,), 'mlpclassifier__activation': 'relu'}
Best test accuracy : 0.7432183908045977


**9) Using the best model found above, report the precision, the recall, the accuracy, the F1 score and the area under the ROC curve. Use a 10 folds cross-validation, and report the means of these metrics across all folds for the training and test folds.**

In [ ]:
best_estimator = grid.best_estimator_

scoring = ['precision', 'recall', 'accuracy', 'f1', 'roc_auc']

cv_results = cross_validate(best_estimator, X, y, cv=10, scoring=scoring, return_train_score=True)

print('Test Precision : {}'.format(cv_results['test_precision'].mean()))
print('Train Precision : {}'.format(cv_results['train_precision'].mean()))
print('Test Recall : {}'.format(cv_results['test_recall'].mean()))
print('Train Recall : {}'.format(cv_results['train_recall'].mean()))
print('Test Accuracy : {}'.format(cv_results['test_accuracy'].mean()))
print('Train Accuracy : {}'.format(cv_results['train_accuracy'].mean()))
print('Test F1 score : {}'.format(cv_results['test_f1'].mean()))
print('Train F1 score : {}'.format(cv_results['train_f1'].mean()))
print('Test AUROC : {}'.format(cv_results['test_roc_auc'].mean()))
print('Train AUROC : {}'.format(cv_results['train_roc_auc'].mean()))

Test Precision : 0.645
Train Precision : 0.7626033967740431
Test Recall : 0.5033333333333333
Train Recall : 0.6394117647058823
Test Accuracy : 0.7467816091954023
Train Accuracy : 0.8196296403796476
Test F1 score : 0.5555950490857611
Train F1 score : 0.6951755621432663
Test AUROC : 0.7637280701754385
Train AUROC : 0.8775512965841662


What do these metrics actually compute to evaluate the performance of our model ? 

* Precision = $\frac{TP}{TP + FP}$
  
  *  Fraction of the points that I **correctly** predicted as positive among all the points that I predicted as positive.  

* Recall = $\frac{TP}{TP + FN}$

  * Fraction of the points that I **correctly** predicted as positive among all the points that I should have predicted as positive. 

* Accuracy = $\frac{TP + TN}{N + P}$

  * Fraction of the points that I correctly predicted. 

* F1-score = $2\frac{Precision * Recall}{Precision + Recall}$

  * Harmonic mean of precision and recall. 

* AUROC : All the models used in this lab (logistic regression, random forest, svc) actually output the probability that a datapoint belongs to the positive class. Given this probability, we must then decide on a threshold above which the point belongs to the positive class. Arbitrarly, this threshold is usually set to 0.5, such that if the probability output by the model is greater or equal to 0.5, the point is classified as "1", and "0"otherwise. However, setting the threshold at 0.5 is rather arbitrary, and other values might be better suited for the task at hand. For instance, if you want to detect credit card fraud, you want to make sure to capture as much fraud as possible, even if that means classifying a couple of legitimate transactions as fraud by doing so. To this end, you might want to lower your threshold, let's say to a value of 0.2, to capture as much fraud as possible. 

  * We can further define the Specificity, or True Negative Rate, as $\frac{TN}{N}$. As the TP, TN, FP, FN are dependent on the selected threshold, so are the Recall and the Specificity.  The Receiver Operating Curve (ROC) actually displays the points $\big(\text{Recall}(\tau), 1-\text{Specificity}(\tau)\big)$ for different values of the threshold $\tau$. The AUROC (Are Under the ROC) is simply the area under the obtained curve, it varies between 0 and 1, 1 being the AUROC of a perfect classifier. 